In [1]:
import os

In [2]:
pwd

'f:\\Projects\\End-to-End-Chest-Cancer-Classification-Using-MLFlow-and-DVC-\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'f:\\Projects\\End-to-End-Chest-Cancer-Classification-Using-MLFlow-and-DVC-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
  root_dir: Path
  trained_model_path: Path
  updated_base_model_path: Path
  training_data: Path
  params_epochs: int
  params_batch_size: int
  params_is_augmentation: bool 
  params_image_size: list

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [8]:
class ConfigurationManager:
  def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)

    create_directories([self.config.artifacts_root])


  def get_model_trainer_config(self) -> ModelTrainerConfig:
    config = self.config.model_trainer
    prepare_base_model = self.config.prepare_base_model
    params = self.params
    training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'Chest-CT-Scan-data')

    create_directories([Path(config.root_dir)])

    model_trainer_config = ModelTrainerConfig(
      root_dir = Path(config.root_dir),
      trained_model_path = Path(config.trained_model_path),
      updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
      training_data = Path(training_data),
      params_epochs = params.EPOCHS,
      params_batch_size = params.BATCH_SIZE,
      params_is_augmentation = params.AUGMENTATION,
      params_image_size = params.IMAGE_SIZE
    )

    return model_trainer_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
class ModelTraining:
  def __init__(self, config: ModelTrainerConfig):
    self.config = config

  def get_base_model(self):
    self.model = tf.keras.models.load_model(
      self.config.updated_base_model_path
    )

  def train_valid_generator(self):
    datagenerator_kwargs = dict(
      rescale=1./255,
      validation_split=0.2
    )

    dataflow_kwargs = dict(
      target_size=self.config.params_image_size[:-1],
      batch_size=self.config.params_batch_size,
      interpolation='bilinear'
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
      **datagenerator_kwargs
    )

    self.valid_datagenerator = valid_datagenerator.flow_from_directory(
      directory=self.config.training_data,
      subset='validation',
      shuffle=False,
      **dataflow_kwargs
    )

    if self.config.params_is_augmentation:
      train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        **datagenerator_kwargs
      )
    else:
      train_datagenerator = valid_datagenerator

    self.train_datagenerator = train_datagenerator.flow_from_directory(
      directory=self.config.training_data,
      subset='training',
      shuffle=True,
      **dataflow_kwargs
    )


  @staticmethod
  def save_model(path: Path, model: tf.keras.Model):
    model.save(path)


  def train(self):
    self.steps_per_epoch = self.train_datagenerator.samples // self.train_datagenerator.batch_size
    self.validation_steps = self.valid_datagenerator.samples // self.valid_datagenerator.batch_size

    self.model.fit(
      self.train_datagenerator,
      epochs=self.config.params_epochs,
      steps_per_epoch=self.steps_per_epoch,
      validation_data=self.valid_datagenerator,
      validation_steps=self.validation_steps
    )

    self.save_model(path=self.config.trained_model_path, model=self.model)

In [11]:
try:
  config = ConfigurationManager()
  model_trainer_config = config.get_model_trainer_config()
  model_training = ModelTraining(config=model_trainer_config)
  model_training.get_base_model()
  model_training.train_valid_generator()
  model_training.train()
except Exception as e:
  raise e

[2024-07-26 18:26:28,158: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-26 18:26:28,170: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-26 18:26:28,172: INFO: common: created directory at artifacts]
[2024-07-26 18:26:28,175: INFO: common: created directory at artifacts\training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
17/17 [==============================] - 105s 6s/step - loss: 11.9177 - accuracy: 0.5637 - val_loss: 18.5807 - val_accuracy: 0.3906
